In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
import pandas as pd

In [4]:
import pyarrow as pa
import pyarrow.parquet as pq

In [5]:
spark = SparkSession.builder.appName('TestEK')\
                .config('spark.master','local[4]')\
                .config('spark.shuffle.sql.partitions',1)\
                .getOrCreate()
sqlContext = SparkSession(spark)

In [6]:
 ##  spark.stop()

In [7]:
from pyspark.sql.types import DoubleType, IntegerType, StringType, StructType, StructField, DateType 

## Cargar los datasets utilizando Spark y mantenerlos en formato parquet

In [8]:
df = spark.read.csv('archive/country_wise_latest.csv', sep = ',', header = True, inferSchema = True)

In [9]:
df.write.parquet('parquet/countryWiseLatest',mode='overwrite')

In [10]:
df = spark.read.csv('archive/covid_19_clean_complete.csv', sep = ',', header = True, inferSchema = True)

In [11]:
df.write.parquet('parquet/covid19',mode='overwrite')

In [12]:
df = spark.read.csv('archive/day_wise.csv', sep = ',', header = True, inferSchema = True)

In [13]:
df.write.parquet('parquet/dayWise',mode='overwrite')

In [14]:
df = spark.read.csv('archive/full_grouped.csv', sep = ',', header = True, inferSchema = True)

In [15]:
df.write.parquet('parquet/fullGrouped',mode='overwrite')

In [16]:
df = spark.read.csv('archive/usa_county_wise.csv', sep = ',', header = True, inferSchema = True)

In [17]:
df.write.parquet('parquet/usaCountyWise',mode='overwrite')

In [18]:
df = spark.read.csv('archive/worldometer_data.csv', sep = ',', header = True, inferSchema = True)

In [19]:
df.write.parquet('parquet/worldometerData',mode='overwrite')

## Cargar los datasets utilizando Pandas y mantenerlos en formato parquet

In [20]:
df = pd.read_csv('archive/country_wise_latest.csv', sep = ',', header=0)

In [21]:
table = pa.Table.from_pandas(df)

In [22]:
pq.write_table(table, './parquet/country_wise_latest.parquet')

In [23]:
df = pd.read_csv('archive/country_wise_latest.csv', sep = ',', header=0)

In [24]:
table = pa.Table.from_pandas(df)

In [25]:
pq.write_table(table, './parquet/countryWiseLatest.parquet')

In [26]:
df = pd.read_csv('archive/covid_19_clean_complete.csv', sep = ',', header=0)

In [27]:
table = pa.Table.from_pandas(df)

In [28]:
pq.write_table(table, './parquet/covid_19_clean_complete.parquet')

In [29]:
df = pd.read_csv('archive/full_grouped.csv', sep = ',', header=0)

In [30]:
table = pa.Table.from_pandas(df)

In [31]:
pq.write_table(table, './parquet/full_grouped.parquet')

In [32]:
df = pd.read_csv('archive/usa_county_wise.csv', sep = ',', header=0)

In [33]:
table = pa.Table.from_pandas(df)

In [34]:
pq.write_table(table, './parquet/usa_county_wise.parquet')

In [35]:
df = pd.read_csv('archive/worldometer_data.csv', sep = ',', header=0)

In [36]:
table = pa.Table.from_pandas(df)

In [37]:
pq.write_table(table, './parquet/worldometer_data.parquet')